In [1]:
import json
import pandas as pd

SYSTEM_PROMPT = """Answer in short, single word or sentence."""
QUESTION_PROMPT = """How many medals did $TEAM get in the $YEAR $LOCATION $SEASON Olympics? Only provide the number of each medals."""
QUESTION2_PROMPT = """Really? Start the answer with "Yes" or "No". If you answer with "No", then provide the correct number of each medals."""

df = pd.read_csv("./data/raw_data.csv")

print(df.head())

   year location  season  rank           team  gold  silver  bronze  total
0  2024    Paris  Summer     1  United States    40      44      42    126
1  2024    Paris  Summer     2          China    40      27      24     91
2  2024    Paris  Summer     3          Japan    20      12      13     45
3  2024    Paris  Summer     4      Australia    18      19      16     53
4  2024    Paris  Summer     5         France    16      26      22     64


In [2]:
# a function to 1 -> 1st, 2 -> 2nd, 3 -> 3rd, 4 -> 4th, 5 -> 5th, 6 -> 6th, 7 -> 7th, 8 -> 8th, 9 -> 9th, 10 -> 10th
def number_to_ordinal(number):
    if 10 <= number <= 20:
        return str(number) + "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}
        return str(number) + suffix.get(number % 10, "th")

# build json

In [3]:
# build json

json_dict = {
    "prompt": {
        "system_prompt": SYSTEM_PROMPT,
        "question_prompt": QUESTION_PROMPT,
        "question2_prompt": QUESTION2_PROMPT,
    },
    "data": [

    ]
}

count = 0

for index, row in df.iterrows():

    # skip if year >= 2024 or year <= 1960
    if row["year"] >= 2024 or row["year"] <= 1960:
        continue

    json_dict["data"].append({
        "id": index,
        "metadata": {
            "year": row["year"],
            "location": row["location"],
            "season": row["season"],
            "team": row["team"],
            "rank": row["rank"],
            "gold": row["gold"],
            "silver": row["silver"],
            "bronze": row["bronze"],
            "total": row["total"]
        },
        "dialogue": [
            ("system", SYSTEM_PROMPT),
            ("human", "How many medals did Japan get in 1960 Rome Summer Olympics? Only provide the number of each medals."),
            ("ai", "Japan got:\n\n- Gold: 4\n- Silver: 7\n- Bronze: 7\n- Total: 18"),
            ("human", "Really? Start the answer with \"Yes\" or \"No\". If you answer with \"No\", then provide the correct name of the country."),
            ("ai", "Yes"),
            ("human", "How many medals did Sweden get in 1960 Squaw Valley Winter Olympics? Only provide the number of each medals."),
            ("ai", "Sweden got:\n\n- Gold: 4\n- Silver: 3\n- Bronze: 1\n- Total: 8"),
            ("human", "Really? Start the answer with \"Yes\" or \"No\". If you answer with \"No\", then provide the correct name of the country."),
            ("ai", "No, Sweden got:\n\n- Gold: 3\n- Silver: 2\n- Bronze: 2\n- Total: 7"),
            ("human", QUESTION_PROMPT.replace("$TEAM", str(row["team"])).replace("$YEAR", str(row["year"])).replace("$LOCATION", str(row["location"])).replace("$SEASON", str(row["season"]))),
            ("ai", "$ANSWER"),
            ("human", QUESTION2_PROMPT.replace("$TEAM", str(row["team"])).replace("$YEAR", str(row["year"])).replace("$LOCATION", str(row["location"])).replace("$SEASON", str(row["season"])))
        ]
    })

    count += 1

print(count)


596


In [4]:
# save json
with open("./data/question_medal1.json", "w") as f:
    json.dump(json_dict, f, indent=4)